In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('nutrition.csv', index_col=['name']).drop(columns=['Unnamed: 0'])

In [3]:
df.head(2)

,serving_size,calories,total_fat,saturated_fat,cholesterol,sodium,choline,folate,folic_acid,niacin,...,fat,saturated_fatty_acids,monounsaturated_fatty_acids,polyunsaturated_fatty_acids,fatty_acids_total_trans,alcohol,ash,caffeine,theobromine,water
name,,,,,,,,,,,,,,,,,,,,,
Cornstarch,100 g,381,0.1g,NaN,0,9.00 mg,0.4 mg,0.00 mcg,0.00 mcg,0.000 mg,...,0.05 g,0.009 g,0.016 g,0.025 g,0.00 mg,0.0 g,0.09 g,0.00 mg,0.00 mg,8.32 g
"Nuts, pecans",100 g,691,72g,6.2g,0,0.00 mg,40.5 mg,22.00 mcg,0.00 mcg,1.167 mg,...,71.97 g,6.180 g,40.801 g,21.614 g,0.00 mg,0.0 g,1.49 g,0.00 mg,0.00 mg,3.52 g


### Single Value Access with .at and .iat

In [4]:
df.loc['Nuts, pecans', 'serving_size']

'100 g'

Using .at we can get similar result like above

In [5]:
df.at['Nuts, pecans', 'serving_size']

'100 g'

In [6]:
print(df.iloc[1,1])
print(df.iat[1,1])

691
691


Purpose using .at and .iat:
- Unlike .loc and .iloc, .at and .iat are only used for accessing single value
- faster compare to .loc and .iloc because the lack of overhead

### The get_loc() Method

Using df.index or df.columns .get_loc('...') to get the position or location of the search

In [7]:
df.columns.get_loc('total_fat')

2

In [8]:
df.index.get_loc('Beef, simmered, cooked, brain, variety meats and by-products')

5935

### Dealing with units ('g', 'mg' etc.)

One way to remove the units is using replace method with regex pattern. But there are more than one unit

Need to add '.astype(str)' so that the int values doesnt included

In [9]:
units = df.astype('str').replace('[^a-zA-z]','', regex=True)
units.head(2)

,serving_size,calories,total_fat,saturated_fat,cholesterol,sodium,choline,folate,folic_acid,niacin,...,fat,saturated_fatty_acids,monounsaturated_fatty_acids,polyunsaturated_fatty_acids,fatty_acids_total_trans,alcohol,ash,caffeine,theobromine,water
name,,,,,,,,,,,,,,,,,,,,,
Cornstarch,g,,g,nan,,mg,mg,mcg,mcg,mg,...,g,g,g,g,mg,g,g,mg,mg,g
"Nuts, pecans",g,,g,g,,mg,mg,mcg,mcg,mg,...,g,g,g,g,mg,g,g,mg,mg,g


Regex:
- [] --> Range definition
- a-zA-Z --> all alphabet a-z and capital A-Z
- ^ all that not found in ... (like tilde notation '~')

So We take all the numeric instances and replace it with ''

Actually we can fix the units in the columns name like 'serving_size' to 'serving_size_g'. It is easier than to convert all the data

But now the problem is there are NaN values in the data. Solution is using mode method - the most frequent value

In [10]:
units = units.mode()

In [11]:
units['serving_size'].at[0]

'g'

We need to add_suffix() the 'g' in the column names

In [12]:
for i in units:
    print(units[i][0])

g

g
g
mg
mg
mg
mcg
mcg
mg
mg
mg
mg
IU
mcg
mcg
mcg
mcg
mcg

mcg
mg
mg
IU
mg
mg
mcg
mg
mg
mg
mg
mg
mg
mg
mcg
mg
g
g
g
g
g
g
g
g

g
g
g
g
g
g
g
g
g
g
g
g
g
g






g
g
g
g
mg
g
g
mg
mg
g


We need to handle the gaps

In [13]:
units = units.replace('', np.NaN).dropna(axis=1)

Now making a mapper dictionary. for example:
mapper = {'serving_size': 'serving_size_g', etc..}

In [14]:
mapper = {i:f'{i}_{units[i][0]}' for i in units}
mapper

{'serving_size': 'serving_size_g',
 'total_fat': 'total_fat_g',
 'saturated_fat': 'saturated_fat_g',
 'cholesterol': 'cholesterol_mg',
 'sodium': 'sodium_mg',
 'choline': 'choline_mg',
 'folate': 'folate_mcg',
 'folic_acid': 'folic_acid_mcg',
 'niacin': 'niacin_mg',
 'pantothenic_acid': 'pantothenic_acid_mg',
 'riboflavin': 'riboflavin_mg',
 'thiamin': 'thiamin_mg',
 'vitamin_a': 'vitamin_a_IU',
 'vitamin_a_rae': 'vitamin_a_rae_mcg',
 'carotene_alpha': 'carotene_alpha_mcg',
 'carotene_beta': 'carotene_beta_mcg',
 'cryptoxanthin_beta': 'cryptoxanthin_beta_mcg',
 'lutein_zeaxanthin': 'lutein_zeaxanthin_mcg',
 'vitamin_b12': 'vitamin_b12_mcg',
 'vitamin_b6': 'vitamin_b6_mg',
 'vitamin_c': 'vitamin_c_mg',
 'vitamin_d': 'vitamin_d_IU',
 'vitamin_e': 'vitamin_e_mg',
 'tocopherol_alpha': 'tocopherol_alpha_mg',
 'vitamin_k': 'vitamin_k_mcg',
 'calcium': 'calcium_mg',
 'copper': 'copper_mg',
 'irom': 'irom_mg',
 'magnesium': 'magnesium_mg',
 'manganese': 'manganese_mg',
 'phosphorous': 'phospho

Finally we can rename the columns with the new mapper

In [15]:
df = df.rename(columns=mapper)
df.head(2)

,serving_size_g,calories,total_fat_g,saturated_fat_g,cholesterol_mg,sodium_mg,choline_mg,folate_mcg,folic_acid_mcg,niacin_mg,...,fat_g,saturated_fatty_acids_g,monounsaturated_fatty_acids_g,polyunsaturated_fatty_acids_g,fatty_acids_total_trans_mg,alcohol_g,ash_g,caffeine_mg,theobromine_mg,water_g
name,,,,,,,,,,,,,,,,,,,,,
Cornstarch,100 g,381,0.1g,NaN,0,9.00 mg,0.4 mg,0.00 mcg,0.00 mcg,0.000 mg,...,0.05 g,0.009 g,0.016 g,0.025 g,0.00 mg,0.0 g,0.09 g,0.00 mg,0.00 mg,8.32 g
"Nuts, pecans",100 g,691,72g,6.2g,0,0.00 mg,40.5 mg,22.00 mcg,0.00 mcg,1.167 mg,...,71.97 g,6.180 g,40.801 g,21.614 g,0.00 mg,0.0 g,1.49 g,0.00 mg,0.00 mg,3.52 g


Last we will remove the units on the values

In [16]:
df.replace('[a-zA-Z]','', regex=True, inplace=True)
df.head(2)

,serving_size_g,calories,total_fat_g,saturated_fat_g,cholesterol_mg,sodium_mg,choline_mg,folate_mcg,folic_acid_mcg,niacin_mg,...,fat_g,saturated_fatty_acids_g,monounsaturated_fatty_acids_g,polyunsaturated_fatty_acids_g,fatty_acids_total_trans_mg,alcohol_g,ash_g,caffeine_mg,theobromine_mg,water_g
name,,,,,,,,,,,,,,,,,,,,,
Cornstarch,100,381,0.1,NaN,0,9.00,0.4,0.00,0.00,0.000,...,0.05,0.009,0.016,0.025,0.00,0.0,0.09,0.00,0.00,8.32
"Nuts, pecans",100,691,72,6.2,0,0.00,40.5,22.00,0.00,1.167,...,71.97,6.180,40.801,21.614,0.00,0.0,1.49,0.00,0.00,3.52


In [17]:
df.dtypes.value_counts()

object    73
int64      2
dtype: int64

### Filtering in 2D

In [18]:
df.filter(like='octopus', axis=0)

,serving_size_g,calories,total_fat_g,saturated_fat_g,cholesterol_mg,sodium_mg,choline_mg,folate_mcg,folic_acid_mcg,niacin_mg,...,fat_g,saturated_fatty_acids_g,monounsaturated_fatty_acids_g,polyunsaturated_fatty_acids_g,fatty_acids_total_trans_mg,alcohol_g,ash_g,caffeine_mg,theobromine_mg,water_g
name,,,,,,,,,,,,,,,,,,,,,
"Mollusks, raw, common, octopus",100,82,1,0.2,48,230.00,65.0,16.00,0.00,2.100,...,1.04,0.227,0.162,0.239,48.00,0.0,1.60,0.00,0.00,80.25
"Mollusks, moist heat, cooked, common, octopus",100,164,2.1,0.5,96,460.00,81.0,24.00,0.00,3.780,...,2.08,0.453,0.324,0.477,96.00,0.0,3.20,0.00,0.00,60.50


In [19]:
df.filter(regex='Octopus', axis=0)

,serving_size_g,calories,total_fat_g,saturated_fat_g,cholesterol_mg,sodium_mg,choline_mg,folate_mcg,folic_acid_mcg,niacin_mg,...,fat_g,saturated_fatty_acids_g,monounsaturated_fatty_acids_g,polyunsaturated_fatty_acids_g,fatty_acids_total_trans_mg,alcohol_g,ash_g,caffeine_mg,theobromine_mg,water_g
name,,,,,,,,,,,,,,,,,,,,,
Octopus (Alaska Native),100,56,0.8,0.2,41,0,0,0,0,2.000,...,0.80,0.200,0,0.200,41.00,0,1.50,0,0,84.00


Even with regex, still upper or lowercase sensitive. We need to user regex with pattern that ignore upper- or lowercse

In [20]:
df.filter(regex='(?i)ctopus', axis=0)

,serving_size_g,calories,total_fat_g,saturated_fat_g,cholesterol_mg,sodium_mg,choline_mg,folate_mcg,folic_acid_mcg,niacin_mg,...,fat_g,saturated_fatty_acids_g,monounsaturated_fatty_acids_g,polyunsaturated_fatty_acids_g,fatty_acids_total_trans_mg,alcohol_g,ash_g,caffeine_mg,theobromine_mg,water_g
name,,,,,,,,,,,,,,,,,,,,,
Octopus (Alaska Native),100,56,0.8,0.2,41,0,0,0,0,2.000,...,0.80,0.200,0,0.200,41.00,0,1.50,0,0,84.00
"Mollusks, raw, common, octopus",100,82,1,0.2,48,230.00,65.0,16.00,0.00,2.100,...,1.04,0.227,0.162,0.239,48.00,0.0,1.60,0.00,0.00,80.25
"Mollusks, moist heat, cooked, common, octopus",100,164,2.1,0.5,96,460.00,81.0,24.00,0.00,3.780,...,2.08,0.453,0.324,0.477,96.00,0.0,3.20,0.00,0.00,60.50


(?i) --> modifier with insensitive case

### Filtering both dimensions

Filtering with index that contains 'brain' insensitive case and columns with unit only 'g'

In [21]:
df.filter(regex='(?i)brain', axis=0).filter(like='_g', axis=1).sort_values(by='protein_g', ascending=False).head(2)

,serving_size_g,total_fat_g,saturated_fat_g,protein_g,alanine_g,arginine_g,aspartic_acid_g,cystine_g,glutamic_acid_g,glycine_g,...,carbohydrate_g,fiber_g,sugars_g,fat_g,saturated_fatty_acids_g,monounsaturated_fatty_acids_g,polyunsaturated_fatty_acids_g,alcohol_g,ash_g,water_g
name,,,,,,,,,,,,,,,,,,,,,
"Lamb, pan-fried, cooked, brain, variety meats and by-products",100,22,5.7,16.97,0.962,1.144,1.416,0.177,2.012,0.804,...,0.00,0.0,0,22.19,5.670,4.020,2.280,0,1.82,60.71
"Veal, pan-fried, cooked, brain, variety meats and by-products",100,17,4,14.48,0.745,0.794,1.228,0.151,1.732,0.635,...,0.00,0.0,0,16.75,3.960,4.210,2.440,0,1.88,68.56


### Using Series between() in the DataFrame

In [22]:
df[df['calories'].between(20,60)].head(2)

,serving_size_g,calories,total_fat_g,saturated_fat_g,cholesterol_mg,sodium_mg,choline_mg,folate_mcg,folic_acid_mcg,niacin_mg,...,fat_g,saturated_fatty_acids_g,monounsaturated_fatty_acids_g,polyunsaturated_fatty_acids_g,fatty_acids_total_trans_mg,alcohol_g,ash_g,caffeine_mg,theobromine_mg,water_g
name,,,,,,,,,,,,,,,,,,,,,
"Eggplant, raw",100,25,0.2,NaN,0,2.00,6.9,22.00,0.00,0.649,...,0.18,0.034,0.016,0.076,0.00,0.0,0.66,0.00,0.00,92.30
"Cauliflower, raw",100,25,0.3,0.1,0,30.00,44.3,57.00,0.00,0.507,...,0.28,0.130,0.034,0.031,0.00,0.0,0.76,0.00,0.00,92.07


.between(x,y) --> both x and y are inclusive

### Min, Max, idx[Min, Max]

min(), max() can be applied to dataframes with axis

In order to use min(), max() etc the df values should be int or float

In [26]:
df = df.astype(float)

In [27]:
df['potassium_mg'].max()

16500.0

In [28]:
df.max(axis=1)

name
Cornstarch                                                                                            381.0
Nuts, pecans                                                                                          691.0
Eggplant, raw                                                                                         229.0
Teff, uncooked                                                                                        429.0
Sherbet, orange                                                                                       144.0
                                                                                                      ...  
Beef, raw, all grades, trimmed to 0" fat, separable lean and fat, boneless, top round roast, round    311.0
Lamb, cooked, separable lean only, composite of trimmed retail cuts, frozen, imported, New Zealand    246.0
Lamb, raw, separable lean and fat, composite of trimmed retail cuts, frozen, imported, New Zealand    277.0
Beef, raw, all grades, 

In [29]:
df['potassium_mg'].idxmax()

'Leavening agents, cream of tartar'

In [30]:
df['potassium_mg'].idxmin()

'PACE, Picante Sauce'

In [31]:
df.loc[df['potassium_mg'].idxmax()]

serving_size_g     100.0
calories           258.0
total_fat_g          0.0
saturated_fat_g      NaN
cholesterol_mg       0.0
                   ...  
alcohol_g            0.0
ash_g               36.8
caffeine_mg          0.0
theobromine_mg       0.0
water_g              1.7
Name: Leavening agents, cream of tartar, Length: 75, dtype: float64

Decades ago our ancentors used to eat diet with potassium sodium ration 16:1. But now our diet is reversed to 1:1,36

In [41]:
df['potassium_mg'].sort_values(ascending=True).head(3)

name
KEEBLER, Honey Wheat Crackers, WHEATABLES    0.0
SILK Vanilla soy yogurt (family size)        0.0
Vitasoy USA Nasoya, Lite Silken Tofu         0.0
Name: potassium_mg, dtype: float64

In order to avoid error division by zero. We will replace 0 with 1

In [32]:
K_Na=(df['potassium_mg'].replace(0,1)/df['sodium_mg'].replace(0,1)).sort_values(ascending=False)

In [42]:
K_Na[K_Na.between(15,20)].sample(3)

name
Fruit juice smoothie, GREEN GOODNESS, BOLTHOUSE FARMS    16.090909
Rice, UNCLE BENS, cooked, parboiled, brown               15.250000
Peppers, unprepared, chopped, frozen, red, sweet         18.200000
dtype: float64

In [36]:
df.nlargest(10,columns='cholesterol_mg').head(2)

,serving_size_g,calories,total_fat_g,saturated_fat_g,cholesterol_mg,sodium_mg,choline_mg,folate_mcg,folic_acid_mcg,niacin_mg,...,fat_g,saturated_fatty_acids_g,monounsaturated_fatty_acids_g,polyunsaturated_fatty_acids_g,fatty_acids_total_trans_mg,alcohol_g,ash_g,caffeine_mg,theobromine_mg,water_g
name,,,,,,,,,,,,,,,,,,,,,
"Beef, simmered, cooked, brain, variety meats and by-products",100.0,151.0,11.0,2.4,3100.0,108.0,490.9,5.0,0.0,3.62,...,10.53,2.394,1.882,1.632,3100.0,0.0,1.46,0.0,0.0,74.86
"Veal, braised, cooked, brain, variety meats and by-products",100.0,136.0,9.6,2.2,3100.0,156.0,0.0,3.0,0.0,2.43,...,9.63,2.180,1.740,1.490,3100.0,0.0,1.40,0.0,0.0,76.89


In [37]:
df.sort_values(by='cholesterol_mg', ascending=False).head(2)

,serving_size_g,calories,total_fat_g,saturated_fat_g,cholesterol_mg,sodium_mg,choline_mg,folate_mcg,folic_acid_mcg,niacin_mg,...,fat_g,saturated_fatty_acids_g,monounsaturated_fatty_acids_g,polyunsaturated_fatty_acids_g,fatty_acids_total_trans_mg,alcohol_g,ash_g,caffeine_mg,theobromine_mg,water_g
name,,,,,,,,,,,,,,,,,,,,,
"Veal, braised, cooked, brain, variety meats and by-products",100.0,136.0,9.6,2.2,3100.0,156.0,0.0,3.0,0.0,2.43,...,9.63,2.180,1.740,1.490,3100.0,0.0,1.40,0.0,0.0,76.89
"Beef, simmered, cooked, brain, variety meats and by-products",100.0,151.0,11.0,2.4,3100.0,108.0,490.9,5.0,0.0,3.62,...,10.53,2.394,1.882,1.632,3100.0,0.0,1.46,0.0,0.0,74.86


In [43]:
df['cholesterol_mg'].nlargest(5)

name
Beef, simmered, cooked, brain, variety meats and by-products          3100.0
Veal, braised, cooked, brain, variety meats and by-products           3100.0
Beef, raw, brain, variety meats and by-products                       3010.0
Lamb, soaked and fried, cooked, brains, imported, New Zealand         2559.0
Pork, braised, cooked, brain, variety meats and by-products, fresh    2552.0
Name: cholesterol_mg, dtype: float64

In [44]:
df.nlargest(2,columns=['cholesterol_mg','total_fat_g'])

,serving_size_g,calories,total_fat_g,saturated_fat_g,cholesterol_mg,sodium_mg,choline_mg,folate_mcg,folic_acid_mcg,niacin_mg,...,fat_g,saturated_fatty_acids_g,monounsaturated_fatty_acids_g,polyunsaturated_fatty_acids_g,fatty_acids_total_trans_mg,alcohol_g,ash_g,caffeine_mg,theobromine_mg,water_g
name,,,,,,,,,,,,,,,,,,,,,
"Beef, simmered, cooked, brain, variety meats and by-products",100.0,151.0,11.0,2.4,3100.0,108.0,490.9,5.0,0.0,3.62,...,10.53,2.394,1.882,1.632,3100.0,0.0,1.46,0.0,0.0,74.86
"Veal, braised, cooked, brain, variety meats and by-products",100.0,136.0,9.6,2.2,3100.0,156.0,0.0,3.0,0.0,2.43,...,9.63,2.180,1.740,1.490,3100.0,0.0,1.40,0.0,0.0,76.89


In [45]:
df.filter(regex='(?i)eggplant', axis=0)['sodium_mg'].sort_values(ascending=False)

name
Eggplant, pickled                                  1674.0
Eggplant, with salt, drained, boiled, cooked        239.0
Eggplant, raw                                         2.0
Eggplant, without salt, drained, boiled, cooked       1.0
Name: sodium_mg, dtype: float64

In [46]:
df.sample(4,axis=0).sample(2, axis=1)

,vitamin_a_IU,sodium_mg
name,,
"DOMINO'S 14"" Pepperoni Pizza, Crunchy Thin Crust",0.0,783.0
"KEEBLER, Shortbread Pie Crust, READY CRUST",0.0,430.0
"Turnip greens, unprepared, frozen",6184.0,12.0
"Cherries, solids and liquids, light syrup pack, canned, red, sour",726.0,7.0


In [47]:
df[df['vitamin_c_mg'].between(20,40)].nsmallest(1, columns='calories')

,serving_size_g,calories,total_fat_g,saturated_fat_g,cholesterol_mg,sodium_mg,choline_mg,folate_mcg,folic_acid_mcg,niacin_mg,...,fat_g,saturated_fatty_acids_g,monounsaturated_fatty_acids_g,polyunsaturated_fatty_acids_g,fatty_acids_total_trans_mg,alcohol_g,ash_g,caffeine_mg,theobromine_mg,water_g
name,,,,,,,,,,,,,,,,,,,,,
"Beverages, fortified with vitamin C, diet, citrus, ready-to-drink, green, tea",100.0,1.0,0.0,NaN,0.0,26.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.08,5.0,0.0,99.62
